FIRST 3 CELLS IS A COMMON MISTAKE, LAST 3 WORK AND INTRODUCE IDEA OF OVERFITTING DATA, NORMILIZING DATA, INCREASING BRAIN POWER DOESN'T WORK LIKE YOU'D THINK (HIDDEN NEURONS), LEARNING RATE, AND BIAS

Helpful Links:
- https://ocw.mit.edu/courses/sloan-school-of-management/15-097-prediction-machine-learning-and-statistics-spring-2012/datasets/digits.csv
- https://www.kdnuggets.com/2017/08/37-reasons-neural-network-not-working.html/2
- https://towardsdatascience.com/understanding-learning-rates-and-how-it-improves-performance-in-deep-learning-d0d4059c1c10
- https://stats.stackexchange.com/questions/338255/what-is-effect-of-increasing-number-of-hidden-layers-in-a-feed-forward-nn
- https://stats.stackexchange.com/questions/352036/what-should-i-do-when-my-neural-network-doesnt-learn
- https://medium.com/@rrfd/standardize-or-normalize-examples-in-python-e3f174b65dfc

In [0]:
import pandas as pd
import numpy as np

In [0]:
# when it imports, the first row is counted as column data (had to do complicated stuff because of this)
dataset = pd.read_csv("https://ocw.mit.edu/courses/sloan-school-of-management/15-097-prediction-machine-learning-and-statistics-spring-2012/datasets/digits.csv")
datasetArray = np.array(dataset)
xData = datasetArray.T[0:-1]
xData = xData.T[0:1]
print(xData.shape)
yData = datasetArray.T[-1]
yData = yData.T[0:xData.shape[0]]
print(yData.shape)

In [0]:
class NeuralNetwork:
    def __init__(self, x, y, hiddenNeurons=25):
        self.input = x
        np.random.seed(1)
        self.weights1 = 2 * np.random.rand(self.input.shape[1],hiddenNeurons) - 1 # 16 x 25
        if len(y.shape) == 2:
          self.weights2 = 2 * np.random.rand(hiddenNeurons,y.shape[1]) - 1   # 25 x 1
        else:
          self.weights2 = 2 * np.random.rand(hiddenNeurons, 1) - 1
        self.y = y
        self.output = np.zeros(self.y.shape)

    def activate(self,x,deriv=False):
        if(deriv==True):
              return x*(1-x)
        return 1/(1+np.exp(-x))
    
    def feedforward(self,x=None):
        if(x is None):
          self.layer1 = self.activate(np.dot(self.input, self.weights1)) # 10991 x 16 * 16 x 25 = 10991 x 25 (layer 1)
        else:
          self.layer1 = self.activate(np.dot(x, self.weights1))
        self.output = self.activate(np.dot(self.layer1, self.weights2)) # 10991 x 25 * 25 x 1 = 10991 x 1 (output)
        
    def backprop(self):
        errorDerivative = 2*(self.y[:,None] - self.output) * activate(self.output, True)
        d_weights2 = np.dot(self.layer1.T, errorDerivative) # 25 x 10991 * 10991 x 1 = 25 x 1 (dweights 2)
        deltaHelp = np.dot(errorDerivative, self.weights2.T) * activate(self.layer1, True)
        d_weights1 = np.dot(self.input.T,  deltaHelp) # 16 x 10991 * 10991 x 25 = 16 x 25 (dweights 1)
        self.weights1 += d_weights1
        self.weights2 += d_weights2
network = NeuralNetwork(xData,yData)
for i in range(6000):
    network.feedforward()
    if(i % 1000 == 0):
      error = np.sum((network.y[:,None] - network.output) ** 2)
      print("Error: " + str(error))
    network.backprop()
toPredict = xData[0]
network.feedforward(toPredict)
print(str(network.output) + " when " + str(toPredict) +  " is predicted compared to " + str(yData[0]))


Error: 58.93607351341663
Error: 49.00009996429573
Error: 49.000049990436146
Error: 49.000033328917034
Error: 49.000024997449714
Error: 49.000019998335006
[0.99999881] when [ 80 100  18  98  60  66 100  29  42   0   0  23  42  61  56  98] is predicted compared to 8


In [0]:
import pandas as pd
import numpy as np
import math
from sklearn import preprocessing as pr

In [0]:
# when it imports, the first row is counted as column data (had to do complicated stuff because of this)
dataset = pd.read_csv("https://ocw.mit.edu/courses/sloan-school-of-management/15-097-prediction-machine-learning-and-statistics-spring-2012/datasets/digits.csv")
datasetArray = np.array(dataset)
xfData = datasetArray.T[0:-1].T
amountOfData = 150
xData = xfData[0:amountOfData]
print(xData.shape)
yfData = datasetArray.T[-1]
yfData = yfData.T[0:xData.shape[0]]
yData = np.zeros((yfData.shape[0], 10))
print(yData.shape)
for i in range(yfData.shape[0]):
  yData[i][yfData[i]] = 1

(150, 16)
(150, 10)


In [0]:
class NeuralNetwork:
    def __init__(self, x, y, hiddenNeurons=50, learningRate=1, useBias=False, norm=False):
      np.random.seed(1)
      # Normalize vs Standardize (I want to use normalization beause implementing standardization is more complicated and I believe this works better)
      # Between 0 and 1 instead of around 0 
      self.norm = norm
      if norm:
        self.input = pr.normalize(x)
      else:
        self.input = x
      self.weights1 = 2 * np.random.rand(self.input.shape[1],hiddenNeurons) - 1 # 16 x h
      if len(y.shape) == 2:
        self.weights2 = 2 * np.random.rand(hiddenNeurons,y.shape[1]) - 1   # h x 8
      else:
        self.weights2 = 2 * np.random.rand(hiddenNeurons, 1) - 1
      self.y = y
      self.output = np.zeros(self.y.shape)
      self.learningRate = learningRate
      self.useBias = useBias
      if useBias:
        self.bias1 = 2 * np.random.rand(hiddenNeurons) - 1
        self.bias2 = 2 * np.random.rand(self.weights2.shape[1]) - 1
    
    def activate(self,x,deriv=False):
      if deriv:
        return x*(1-x)
      return 1/(1+np.exp(-x))    
    
    def feedforward(self,x=None,index=0):
      # To Hidden
      if x is None:
        if self.useBias:
          temp = np.dot(self.input, self.weights1)
          for i in range(temp.shape[0]):
            temp[i] += self.bias1
          self.layer1 = self.activate(temp)    
        else:  
          self.layer1 = self.activate(np.dot(self.input, self.weights1)) # amount of data (aod) x 16 * 16 x h = aod x h (layer 1)
      else:
        if self.norm:
          x = pr.normalize(x)
        if self.useBias:
          self.layer1 = self.activate(np.dot(x[index], self.weights1) + self.bias1)    
        else:  
          self.layer1 = self.activate(np.dot(x[index], self.weights1))

      # To Output
      if x is None:
        if self.useBias:
          temp = np.dot(self.layer1, self.weights2)
          for i in range(temp.shape[0]):
            temp[i] += self.bias2
          self.output = self.activate(temp)    
        else:
          self.output = self.activate(np.dot(self.layer1, self.weights2)) # aod x h * h x 8 = aod x 8 (output)
      else:
        if self.useBias:
          self.output = self.activate(np.dot(self.layer1, self.weights2) + self.bias2)    
        else:
          self.output = self.activate(np.dot(self.layer1, self.weights2))
        
    def backprop(self):
      deltaHidden = 2*(self.y - self.output) * activate(self.output, True) # aod x 8
      d_weights2 = np.dot(self.layer1.T,deltaHidden) # h x aod * aod x 8 = h x 8 (dweights 2)
      deltaInput = np.dot(deltaHidden,self.weights2.T) * activate(self.layer1, True) # aod x h
      d_weights1 = np.dot(self.input.T,deltaInput) # 16 x aod * aod x h = 16 x h (dweights 1)
      self.weights1 += d_weights1 * self.learningRate
      self.weights2 += d_weights2 * self.learningRate
      if self.useBias:
        d_bias2 = np.sum(deltaHidden,axis = 0,keepdims = True)[0]
        d_bias1 = np.sum(deltaInput,axis = 0,keepdims = True)[0]
        self.bias1 += d_bias1 * self.learningRate
        self.bias2 += d_bias2 * self.learningRate

def checkOutput(output):
  best = 0
  for i in range(len(output)):
    if output[best] < output[i]:
      best = i
  return best
network = NeuralNetwork(xData,yData,hiddenNeurons=20,learningRate=.01,useBias=True, norm=True)
for i in range(60001):
    network.feedforward()
    if(i % 10000 == 0):
      error = np.sum((network.y - network.output) ** 2)
      print("Error: " + str(error))
    network.backprop()
X = xfData
for i in range(0, 10, 2):
  network.feedforward(X, i)
  newOut = np.zeros((1, 10))
  newOut[0][datasetArray.T[-1][i]] = 1
  print(str(network.output)) 
  print(" aka: " + str(checkOutput(network.output)) + " when " + str(xfData[i]) +  " is predicted compared to " + str(newOut[0]) + " aka: " + str(datasetArray.T[-1][i]))
  print("Prediction Correct?: " + str(checkOutput(network.output) == datasetArray.T[-1][i]))
  print("----------------------------------------")

counterCorrect = 0
counterTotal = X.shape[0]
for i in range(amountOfData,counterTotal):
  network.feedforward(X, i)
  if checkOutput(network.output) == datasetArray.T[-1][i]:
    counterCorrect += 1
print("Correct Percentage: " + str((counterCorrect/counterTotal) * 100) + "% for the rest of the " + str(counterTotal - amountOfData) + " entries")

Error: 540.930854335504
Error: 11.153001419076478
Error: 10.377123899726339
Error: 10.208766315236959
Error: 10.140112558922313
Error: 10.103845510602586
Error: 10.081747343929894
[9.87050857e-06 7.31775367e-09 2.86942996e-12 3.90570013e-10
 2.45322296e-12 6.03682542e-05 2.34001584e-04 1.16474893e-02
 9.95274391e-01 6.84760223e-11]
 aka: 8 when [ 80 100  18  98  60  66 100  29  42   0   0  23  42  61  56  98] is predicted compared to [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.] aka: 8
Prediction Correct?: True
----------------------------------------
[2.34595474e-04 1.11569037e-03 8.31390361e-11 1.74936898e-04
 4.11747433e-05 4.60249644e-12 9.47247973e-05 2.06578110e-10
 3.09327088e-08 9.99678682e-01]
 aka: 9 when [ 95  82  71 100  27  77  77  73 100  80  93  42  56  13   0   0] is predicted compared to [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.] aka: 9
Prediction Correct?: True
----------------------------------------
[3.22491047e-11 9.94869839e-01 4.80047768e-08 8.64637846e-09
 9.32670110e-06 4.31120324e-07 